<a href="https://colab.research.google.com/github/scorepia/Realtime6005_Project/blob/main/DADS6005_Project_producer2_COKE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance --upgrade --no-cache-dir

!pip install confluent_kafka

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 KB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 KB 142.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 105.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 148.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 84.6 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
 

Importing yahoo finance API

In [2]:
import yfinance as yf

Dumping PEP stock data

In [3]:
ck = yf.download("COKE", period = "1y" , interval = "1d")

[*********************100%***********************]  1 of 1 completed


In [4]:
print(type(ck))

ck.info()
ck.head()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2022-01-13 to 2023-01-12
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Open       251 non-null    float64
 1   High       251 non-null    float64
 2   Low        251 non-null    float64
 3   Close      251 non-null    float64
 4   Adj Close  251 non-null    float64
 5   Volume     251 non-null    int64  
dtypes: float64(5), int64(1)
memory usage: 13.7 KB


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-01-13,607.469971,638.000000,606.200012,621.789978,620.559326,47800
2022-01-14,618.979980,618.979980,599.989990,615.599976,614.381531,20700
2022-01-18,611.929993,611.929993,589.919983,592.080017,590.908142,27900
2022-01-19,598.450012,604.369995,574.530029,576.549988,575.408875,30000
2022-01-20,577.010010,599.919983,576.669983,577.030029,575.887939,22800


In [5]:
ck1 = ck.reset_index()
ck1 = ck1.rename(columns = {'Close':'Close_Price'})
ck1 = ck1[['Date','Close_Price','Volume']]

ck1.info()
ck1.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   Date         251 non-null    datetime64[ns]
 1   Close_Price  251 non-null    float64       
 2   Volume       251 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 6.0 KB


,Date,Close_Price,Volume
0,2022-01-13,621.789978,47800
1,2022-01-14,615.599976,20700
2,2022-01-18,592.080017,27900
3,2022-01-19,576.549988,30000
4,2022-01-20,577.030029,22800


Preparing data for Producer

In [6]:
Lines = []
n = 250

for i in range(n+1):
    j = ck1.loc[i]
    #j = j.to_frame()
    js_data = j.to_json()
    bt_data = js_data.encode()
    Lines.append(js_data)

print(Lines)

['{"Date":1642032000000,"Close_Price":621.7899780273,"Volume":47800}', '{"Date":1642118400000,"Close_Price":615.5999755859,"Volume":20700}', '{"Date":1642464000000,"Close_Price":592.0800170898,"Volume":27900}', '{"Date":1642550400000,"Close_Price":576.549987793,"Volume":30000}', '{"Date":1642636800000,"Close_Price":577.0300292969,"Volume":22800}', '{"Date":1642723200000,"Close_Price":575.9899902344,"Volume":30700}', '{"Date":1642982400000,"Close_Price":600.5100097656,"Volume":37500}', '{"Date":1643068800000,"Close_Price":598.4000244141,"Volume":37000}', '{"Date":1643155200000,"Close_Price":584.9199829102,"Volume":34200}', '{"Date":1643241600000,"Close_Price":566.8499755859,"Volume":23200}', '{"Date":1643328000000,"Close_Price":576.7100219727,"Volume":17100}', '{"Date":1643587200000,"Close_Price":573.0,"Volume":27700}', '{"Date":1643673600000,"Close_Price":566.9600219727,"Volume":37700}', '{"Date":1643760000000,"Close_Price":563.8099975586,"Volume":39700}', '{"Date":1643846400000,"Close

Sending to AWS

In [7]:
from confluent_kafka import Producer
import time
import json
from uuid import uuid4

p = Producer({'bootstrap.servers': 'ec2-13-229-46-113.ap-southeast-1.compute.amazonaws.com:9092'})

def delivery_report(err, msg):
    if err is not None:
        print('Message delivery failed: {}'.format(err))
    else:
        print('Message delivered to {}'.format(msg.value().decode('utf-8')))

for line in Lines:
    p.poll(0)
    sendMsg = line #line.encode().decode('utf-8').strip('\n')
    p.produce('producer2', key=str(uuid4()), value=sendMsg , callback=delivery_report)
    '''time.sleep(1)'''

p.flush()

Message delivered to {"Date":1642032000000,"Close_Price":621.7899780273,"Volume":47800}
Message delivered to {"Date":1642118400000,"Close_Price":615.5999755859,"Volume":20700}
Message delivered to {"Date":1642464000000,"Close_Price":592.0800170898,"Volume":27900}
Message delivered to {"Date":1642550400000,"Close_Price":576.549987793,"Volume":30000}
Message delivered to {"Date":1642636800000,"Close_Price":577.0300292969,"Volume":22800}
Message delivered to {"Date":1642723200000,"Close_Price":575.9899902344,"Volume":30700}
Message delivered to {"Date":1642982400000,"Close_Price":600.5100097656,"Volume":37500}
Message delivered to {"Date":1643068800000,"Close_Price":598.4000244141,"Volume":37000}
Message delivered to {"Date":1643155200000,"Close_Price":584.9199829102,"Volume":34200}
Message delivered to {"Date":1643241600000,"Close_Price":566.8499755859,"Volume":23200}
Message delivered to {"Date":1643328000000,"Close_Price":576.7100219727,"Volume":17100}
Message delivered to {"Date":1643

0